In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
from sentence_transformers import SentenceTransformer

In [ ]:
COHERE_API_KEY = os.environ.get("COHERE_API_KEY", "")

EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
model_emb = SentenceTransformer(EMBEDDING_MODEL)

In [ ]:
def augment_prompt(
        query: str,
        user_style: str,
        context: str,
        model: SentenceTransformer = SentenceTransformer(EMBEDDING_MODEL),
        index=None,
) -> str:

    results = [float(val) for val in list(model.encode(query))]

    # get top 10 results from knowledge base
    query_results = index.query(
        vector=results,
        top_k=5,
        include_values=True,
        include_metadata=True
    )['matches']
    text_matches = [match['metadata']['answer'] for match in query_results]


    # get the text from the results
    answers = "\n\n".join(text_matches)
    

    # feed into an augmented prompt
    improved_prompt = f"""
    
      You write WhatsApp replies *exactly as the user would*.

      Your job: given a new incoming message, write the reply the user is most likely to send.

      You are given:
      1) **query** - the new incoming message you must answer.
      2) **similar_past_answers** – real replies the user wrote in the past to similar messages.  
         Use them for tone, vibe, typical phrasing, emojis, and attitude.
      3) **user_style_examples** – random messages the user wrote in other chats.  
         Use them to mimic writing style, vocabulary, length, energy level, and emoji habits.
      4) **recent_context** – the recent messages in this same chat (both sides).  
         Your reply must fit naturally after this context.

      ### Rules:
      - Write the reply **as the user**, in first person.
      - Match the **language**, **tone**, and **emotion** of the conversation.
      - Keep it natural for WhatsApp: short to medium length, can include emojis.
      - If the query contains multiple questions – answer all.
      - If necessary info is missing – ask a short clarifying question.
      - **Never** mention examples, past messages, embeddings, or that you're an AI.
      - **Only output the final WhatsApp-style reply. No explanations.**

      ---

      ### query:
      {query}

      ### similar_past_answers for similar queries:
      {answers}

      ### user_style_examples:
      {user_style}

      ### recent_context:
      {context}
      """


    return improved_prompt, answers

In [ ]:
query = "i need help with my students, did you taught them already the embeddings ppt?"
barbara_messages = build_user_style(kb_df_all, user_id='u_barbara', k=100)[0]
maayan_messages = build_user_style(kb_df_all, user_id='u_maayan', k=100)[0]
users_style ={'u_barbara': barbara_messages, 'u_maayan': maayan_messages}


In [ ]:

augmented_prompt, source_knowledge = augment_prompt(query, user_style, context,model=model_emb,index=index)
co = cohere.Client(api_key=COHERE_API_KEY)
response = co.chat(
        model='command-a-03-2025',
        message=augmented_prompt,
    )
response.text